In [ ]:
!pip install -r ../requirements.txt 

In [ ]:
import torch
from datasets import load_dataset
from langchain_huggingface import HuggingFaceEmbeddings

# Import từ src
from src.config import DATASET_NAME, EMBEDDING_MODEL_NAME, CHROMA_PATH
from src.vector_store import prepare_documents, build_vector_db


# 1. LOAD DATA
print(f"Đang tải dataset: {DATASET_NAME}...")
# Dùng split 'train' để xây dựng kho tri thức (Knowledge Base)
ds = load_dataset(DATASET_NAME, "pqa_artificial", split="train")

# (Tùy chọn) Lấy mẫu nhỏ để test code nếu cần
# ds = ds.select(range(1000)) 

# 2. PREPARE DOCUMENTS
# Chuyển đổi định dạng dataset sang LangChain Document
docs = prepare_documents(ds)

# 3. SETUP EMBEDDING MODEL
print(f"Đang tải model embedding: {EMBEDDING_MODEL_NAME}...")
device = "cuda" if torch.cuda.is_available() else "cpu"
embedding_model = HuggingFaceEmbeddings(
    model_name=EMBEDDING_MODEL_NAME,
    model_kwargs={'device': device}
)

# 4. BUILD & SAVE VECTOR DB
# Hàm này sẽ tự động tạo thư mục chroma_db_storage và lưu index
vector_db = build_vector_db(
    documents=docs,
    embedding_model=embedding_model,
    persist_path=CHROMA_PATH,
    batch_size=2000
)

print(f"Kiểm tra thử: Database hiện có {vector_db._collection.count()} vectors.")